# MS Calculations

Notebook to crunch numbers for the MS.

by Cascade Tuholske 2020.02.23 

Updated 2020.08.27 - CPT  
Updated July/Aug 2021

In [1]:
#### Depdencies 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import glob
import matplotlib.dates as mdates
import os

In [2]:
#### Regressions, no intercept addition is needed because we're using SK LEARN HERE 

def lm_func(df, col):
    
    "simple linear model of a time series data, returns coef"
    
    # Get Data
    X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
    Y_stats = np.array(df.groupby('year')[col].sum()).reshape((-1, 1))

    # Add Intercept
    X_year_2 = sm.add_constant(X_year)

    # Regress
    model = sm.OLS(Y_stats, X_year_2).fit() 
        
    coef = int(model.params[1])
    #coef = int(coef)
            
    # R2 and P
    r2 = model.rsquared_adj
    p = model.pvalues[0]
    
    return coef, round(r2, 2), round(p, 3)

In [5]:
#### Load Data
DATA = 'wbgtmax30' # UPDATE 

# file paths
DATA_IN = "/home/CHIRTS/UEH-daily/stats/"  
FIG_OUT = "/home/cascade/projects/UrbanHeat/figures/"
FN_IN = os.path.join(DATA_IN,DATA+'_EXP.json')
HI_STATS = pd.read_json(FN_IN, orient = 'split')

# Set scale
scale = 10**9


In [6]:
# Drop cites where 1983 had 1 day and none elsewhere

print(len(HI_STATS))
only83 = HI_STATS.groupby('ID_HDC_G0')['tot_days'].sum() == 1 # sum up total days and find those with 1 day
only83 = list(only83[only83 == True].index) # make a list of IDs
sub = HI_STATS[HI_STATS['ID_HDC_G0'].isin(only83)] # subset those IDs
bad_ids = sub[(sub['year'] == 1983) & (sub['tot_days'] == 1)] # drop those from 1983 only
drop_list = list(bad_ids['ID_HDC_G0']) # make a list
HI_STATS= HI_STATS[~HI_STATS['ID_HDC_G0'].isin(drop_list)] # drop those from the list
print(len(HI_STATS))

303518
303178


In [7]:
#### Add In Meta Data (e.g. geographic data)
meta_fn = os.path.join('/home/cascade/projects/UrbanHeat/data/interim/','GHS-UCDB-IDS.csv')
meta_data = pd.read_csv(meta_fn)

#### Merge in meta
HI_STATS = HI_STATS.merge(meta_data, on = 'ID_HDC_G0', how = 'left')

In [8]:
 HI_STATS

,ID_HDC_G0,year,tot_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop,CTR_MN_NM,UC_NM_MN,GCPNT_LAT,GCPNT_LON,region,sub-region,intermediate-region
0,18,2006,2,622283.428656,494664.495616,661553.577433,1.244567e+06,989328.991233,255237.866079,United States,Concord [USA],37.913946,-122.048318,Americas,Northern America,Northern America
1,22,2006,2,70915.650551,52064.452435,73006.671133,1.418313e+05,104128.904869,37702.396233,United States,Livermore [USA],37.688409,-121.753980,Americas,Northern America,Northern America
2,26,2006,3,247944.032154,194088.886834,268055.635628,7.438321e+05,582266.660503,161565.435959,United States,Antioch [USA],37.985433,-121.797516,Americas,Northern America,Northern America
3,27,2006,2,90805.062622,80540.779940,93335.494324,1.816101e+05,161081.559880,20528.565365,United States,Fairfield [USA],38.264013,-122.030253,Americas,Northern America,Northern America
4,28,1998,1,68062.052369,59320.971209,91449.606255,6.806205e+04,59320.971209,8741.081160,United States,Tracy [USA],37.730079,-121.431413,Americas,Northern America,Northern America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303173,13135,2010,0,NaN,132501.788370,211650.696320,0.000000e+00,0.000000,0.000000,Fiji,Suva [FJI],-18.104802,178.461255,Oceania,Melanesia,Melanesia
303174,13135,2011,0,NaN,132501.788370,211650.696320,0.000000e+00,0.000000,0.000000,Fiji,Suva [FJI],-18.104802,178.461255,Oceania,Melanesia,Melanesia
303175,13135,2012,0,NaN,132501.788370,211650.696320,0.000000e+00,0.000000,0.000000,Fiji,Suva [FJI],-18.104802,178.461255,Oceania,Melanesia,Melanesia
303176,13135,2013,0,NaN,132501.788370,211650.696320,0.000000e+00,0.000000,0.000000,Fiji,Suva [FJI],-18.104802,178.461255,Oceania,Melanesia,Melanesia


In [9]:
#### Check to num days per year greatest > 32
check = HI_STATS.sort_values('tot_days', ascending = False)
check[['CTR_MN_NM', 'UC_NM_MN', 'year', 'tot_days']].head(5)

,CTR_MN_NM,UC_NM_MN,year,tot_days
52805,Eritrea,Mits'Iwa [ERI],2016,255
52791,Eritrea,Mits'Iwa [ERI],2002,254
52804,Eritrea,Mits'Iwa [ERI],2015,248
52799,Eritrea,Mits'Iwa [ERI],2010,244
52792,Eritrea,Mits'Iwa [ERI],2003,243


# Global Trends

In [10]:
#### Total Change in people Days
data = HI_STATS.groupby('year')['people_days'].sum()
year = str(data.index[33])
value = str(data.values[33]/10**9)
print('person days in 2016 was '+value+' billion')

year = str(data.index[0])
value = str(data.values[0]/10**9)
print('person days in 1983 was '+value+' billion')

#### Pct Change in Poeple Days 1983 - 2016
pdays16 = data.iloc[len(data) -1]
pdays83 = data.iloc[0]
out = (data.iloc[len(data) -1] - data.iloc[0]) / data.iloc[0] * 100
print('pct increase in people days 83 - 16 is ', out)


person days in 2016 was 118.6608238905162 billion
person days in 1983 was 39.691073649233786 billion
pct increase in people days 83 - 16 is  198.960982862722


In [11]:
#### Rate of change
data = HI_STATS
coef, r2, p = lm_func(data, 'people_days')
print('annual increase in people days ', 'was', coef/10**9, ' p=', p)
coef1, r21, p1 = lm_func(data, 'people_days_heat')
print('annual increase in people days heat ', 'was', coef1/10**9, ' p=', p)
coef2, r22, p2 = lm_func(data, 'people_days_pop')
print('annual increase in people days pop ', 'was', coef2/10**9, ' p=', p)
print('attrib heat ', 'was', coef1 / coef *100, ' p=', p, '\n')

annual increase in people days  was 2.108968631  p= 0.0
annual increase in people days heat  was 0.716240045  p= 0.0
annual increase in people days pop  was 1.392728586  p= 0.0
attrib heat  was 33.96162628840922  p= 0.0 



In [12]:
#### Pct Pday Annual Increase from Heat
coef_pdays, r2_pdays, p_pdays = lm_func(HI_STATS, 'people_days') # regress pdays
coef_heat, r2_heat, p_heat = lm_func(HI_STATS, 'people_days_heat') # regreas heat

print('warming is what pct of total?', coef_heat/coef_pdays *100)

warming is what pct of total? 33.96162628840922


In [13]:
#### Pct heat vs pop
coef_pop, r2_pdays, p_pdays = lm_func(HI_STATS, 'people_days_pop') # regress pdays
coef_heat, r2_heat, p_heat = lm_func(HI_STATS, 'people_days_heat') # regreas heat

print('pct of heat vs pop', coef_heat/coef_pop *100)

pct of heat vs pop 51.42710878485551


# City-level

#### Largest cities compared to global total

In [15]:
#### Top cities
cities_fn = os.path.join('/home/CHIRTS/UEH-daily/ms-tables/wbgtmax30_EXP-TOP50.csv')
cities = pd.read_csv(cities_fn)

In [16]:
top = cities.sort_values('coef_pdays', ascending = False).head(25) # get the top ten cities


In [17]:
# What pct of the global annual increase comes from the top 25 cities?
ans = top['coef_pdays'].sum() / coef # coef comes from rate of change cell
print('Top 25 cities of total annual increase', ans * 100)

Top 25 cities of total annual increase 24.658691183911717


#### Pdays

In [19]:
city_coefs_fn = os.path.join('/home/CHIRTS/UEH-daily/stats/wbgtmax30_TREND_PDAYS05.json')
city_coefs = pd.read_json(city_coefs_fn, orient = 'split')
GHS = gpd.read_file('/home/cascade/projects/UrbanHeat/data/raw/GHS_UCDB/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp')

In [20]:
len(city_coefs)

5985

In [21]:
#### Number of cities w/ sig increase in exposure?
print('The pct of cities w/ increases in exposure: ', len(city_coefs)/len(GHS)*100)


The pct of cities w/ increases in exposure:  45.56528359345261


In [22]:
len(city_coefs)

5985

In [23]:
ans = len(city_coefs[(city_coefs['GCPNT_LAT'] < 23.5) & (city_coefs['GCPNT_LAT'] > -23.5)]) / len(city_coefs)
print('what pct of pday cities are low lat?', ans*100)

what pct of pday cities are low lat? 49.2063492063492


In [24]:
print('what pct of global pop are cities with sig pdays?')

what pct of global pop are cities with sig pdays?


In [25]:
def country_search(country, data_set):
    "what pct of cities had a p-day increase?"
    print('Num of Cities in '+country+' ', len(data_set[data_set['CTR_MN_NM'] == country]) / len(GHS[GHS['CTR_MN_NM'] == country]) *100)

In [26]:
data_set = city_coefs

In [27]:
country_search('Senegal', data_set)

Num of Cities in Senegal  96.96969696969697


In [28]:
country_search('Nigeria', data_set)

Num of Cities in Nigeria  87.99171842650104


In [29]:
country_search('India', data_set)

Num of Cities in India  85.09852216748769


#### Pct of global population exposured

In [31]:
city_coefs_fn = os.path.join('/home/CHIRTS/UEH-daily/stats/wbgtmax30_TREND_PDAYS05.json')
city_coefs = pd.read_json(city_coefs_fn, orient = 'split')

In [32]:
pop_fn = os.path.join('/home/cascade/projects/UrbanHeat/data/interim/','GHS-UCDB-Interp.csv')
pop = pd.read_csv(pop_fn)

p16 = pop[['ID_HDC_G0', 'P2016']]

In [33]:
len(p16)

13135

In [34]:
pdays_pop = pd.merge(city_coefs[['ID_HDC_G0']], p16, on = 'ID_HDC_G0', how = 'inner')

In [35]:
ans = pdays_pop['P2016'].sum() / p16['P2016'].sum() * 100 
print('What is the global urban population in 2016', p16['P2016'].sum() / 10**9)
print('How many people live in cities with increasing exp in 2016', pdays_pop['P2016'].sum() / 10**9)
print('What pct of total urban pop has sig increase exp in 2015', ans)

What is the global urban population in 2016 3.5353262985424414
How many people live in cities with increasing exp in 2016 1.6646639187583419
What pct of total urban pop has sig increase exp in 2015 47.08657074863659


In [36]:
# From UN-DESA 2018 estimates for total global pop in 2015
ans =  pdays_pop['P2016'].sum() / 7383009000 * 100
print('What pct of total world pop has sig increase exp in 2016', ans)

What pct of total world pop has sig increase exp in 2016 22.54722862667974


In [37]:
# UN-DESA Urban pop in 2015 was  3 981 498
p16['P2016'].sum()

3535326298.5424414

#### Total Heat Days 

In [38]:
city_totdays_fn = os.path.join('/home/CHIRTS/UEH-daily/stats/wbgtmax30_TREND_HEATP05.json')
city_totdays = pd.read_json(city_totdays_fn, orient = 'split')

In [39]:
print('What pct of all cities had sig increase in days/yr > WBGT 30 C ?')
print(len(city_totdays)/len(GHS))
print(len(city_totdays))

What pct of all cities had sig increase in days/yr > WBGT 30 C ?
0.39101636848115723
5136


In [40]:
print('What pct of all cities >1 day / yr in days/yr > WBGT 30 C ?')
print(len(city_totdays[city_totdays['coef_totDays'] >= 1])/len(GHS))
print(len(city_totdays[city_totdays['coef_totDays'] >= 1]))

What pct of all cities >1 day / yr in days/yr > WBGT 30 C ?
0.1714503235629996
2252


In [41]:
## How many cities day increase per year ... 1, 3
top = len(city_totdays)
bottom = len(city_totdays[city_totdays['coef_totDays'] >= 2])

In [42]:
print(top)
print(bottom)

5136
24


In [43]:
## What are some big cities (>1m people)?
hot1m = city_totdays[(city_totdays['coef_totDays'] >= 1.5) & (city_totdays['P2016'] >= 10**6)][['coef_totDays', 'UC_NM_MN', 'P2016']].sort_values('coef_totDays')

In [44]:
len(hot1m)

21

In [45]:
hot1m

,coef_totDays,UC_NM_MN,P2016
5603,1.519328,Khulna [BGD],1.255282e+06
3690,1.522995,Hardoi [IND],1.185764e+06
2099,1.524675,Baghdad [IRQ],5.367305e+06
4706,1.534912,Muzaffarpur [IND],1.435602e+06
2059,1.548358,Makkah (Mecca) [SAU],1.312911e+06
1567,1.568678,N'Djamena [TCD]; Kousseri [CMR],1.775332e+06
4707,1.602139,Patna [IND],2.771973e+06
2314,1.612987,Ad-Dammam [SAU],1.564432e+06
5469,1.633461,Bogra [BGD],1.284400e+06
2132,1.659587,Najaf [IRQ],1.012533e+06


In [ ]:
2.162029e+07 / 10**6


#### Dehli & Kolkata 

In [46]:
# Delhi 6955 & Kolkata 9691
K = city_coefs[city_coefs['ID_HDC_G0']== 9691]
D = city_coefs[city_coefs['ID_HDC_G0']== 6955]

In [47]:
print('Share of heat Kolkata', K.coef_heat / K.coef_pdays * 100)

Share of heat Kolkata 5434    52.306758
dtype: float64


In [48]:
print('Share of heat Delhi', D.coef_heat / D.coef_pdays * 100)

Share of heat Delhi 3058    25.869643
dtype: float64


#### Populations of specific cities

In [ ]:
pop = pd.read_csv(DATA_IN+'interim/GHS-UCDB-Interp.csv')

In [ ]:
# 9691, Kolkata 1998
# 2046, Paris 2003
# 4417, Aleppo 2010

In [ ]:
ans = pop[pop['ID_HDC_G0'] == 9691]['P2015'] / 10**3
print('Pop of Kolkata in 1998', ans)

## WBGT 32 vs 28

In [126]:
wbgt32 = pd.read_json('/home/CHIRTS/UEH-daily/stats/wbgtmax32_TREND_PDAYS05.json', orient = 'split')
wbgt30 = pd.read_json('/home/CHIRTS/UEH-daily/stats/wbgtmax30_TREND_PDAYS05.json', orient = 'split')
wbgt28 = pd.read_json('/home/CHIRTS/UEH-daily/stats/wbgtmax28_TREND_PDAYS05.json', orient = 'split')

In [127]:
print('how many cities wbgt 32?', len(wbgt32))
print('how many cities wbgt 32?', len(wbgt30))
print('how many cities wbgt 28?', len(wbgt28))
print('no trend', len(meta_data) - len(wbgt28))
print('dif is', 8510 - 6022)

how many cities wbgt 32? 2979
how many cities wbgt 32? 5985
how many cities wbgt 28? 7628
no trend 5507
dif is 2488


# Regional Trends

In [ ]:
#### Annual Rates

scale = 10**6
geog = 'sub-region'

for label in np.unique(HI_STATS[geog]):
    label = label
    data = HI_STATS[HI_STATS[geog] == label]
    
    #### Rate of change
    coef, r2, p = lm_func(data, 'people_days')
    print('annual increase in people days '+label, 'was', coef/scale, ' p=', p)
    coef1, r21, p1 = lm_func(data, 'people_days_heat')
    print('annual increase in people days heat '+label, 'was', coef1/scale, ' p=', p)
    coef2, r22, p2 = lm_func(data, 'people_days_pop')
    print('annual increase in people days pop '+label, 'was', coef2/scale, ' p=', p)
    print('attrib heat '+label, 'was', coef1 / coef *100, ' p=', p, '\n')
  

In [ ]:
#### Trends for Africa, N & SS
geog = 'region'
location = 'Africa'
data = HI_STATS[HI_STATS[geog] == location]
print(location)

#### Total Change in people Days
data = data.groupby('year')['people_days'].sum()
year = str(data.index[33])
value = str(data.values[33]/10**9)
print('person days in 2016 was '+value+' billion')

year = str(data.index[0])
value = str(data.values[0]/10**9)
print('person days in 1983 was '+value+' billion')

#### Pct Change in Poeple Days 1983 - 2016
pdays16 = data.iloc[len(data) -1]
pdays83 = data.iloc[0]
out = (data.iloc[len(data) -1] - data.iloc[0]) / data.iloc[0] * 100
print('pct increase in people days 83 - 16 is ', out)



In [ ]:
#### S Asia as pct of total  global = 5.245146271 B 

print('pct of total pdays from S Asia is ', 1899.70765 / 10**3 / 5.245146271 * 100)

In [ ]:
#### Median Slope
region = 'Europe'
col = 'coef_heat'
geog = 'region'
scale = 10**3
result = city_coefs[city_coefs[geog]== region][col].median()
print(region, col, 'is ', result/scale)

# Heat Waves

- 9691 Kolkata 1998
- 2046 Paris 2003
- 4417, Aleppo 2010

In [49]:
def make_data(dir_in, geog, location):
    """Function makes data to plot daily city-level HI Max and average
    Args:
        dir_in = directory to get data
        geog = column for geography, city-level = 'ID_HDC_G0'
        location = usually a city id
    """
    
    fn_list = sorted(glob.glob(dir_in+'*.csv')) # get data
    df_out = pd.DataFrame() # to write dataframe
    
     # get leap year cols from 2016
    hi16 = pd.read_csv(fn_list[33]) 
    cols = list(hi16.iloc[:,9:].columns)
    cols = [year[5:] for year in cols] # cols for data frame
    
    temp_list = [] # empty list for temps
    
    # loop through dir and get data
    for i, fn in enumerate(fn_list):
        df = pd.read_csv(fn) # open data frame
        year_label = [(df.columns[9]).split('.')[0]] # get year
        row = df[df[geog] == location]
        temp = row.iloc[:,9:] # get only temp columns
        
        # add in col for leap years
        if temp.shape[1] == 365:
            temp.insert(loc = 59, column = year_label[0]+'.02.29', value = np.nan, allow_duplicates=False)

        # Set Index & Columns
        temp.index = year_label
        temp.columns = cols # revalue to m.d
    
        # add to list
        temp_list.append(temp)
    
    df_out = pd.concat(temp_list) # make one big dataframe
    
    return df_out

def plot_data(df, year, start, end):#, start, end):
    """ Make the data for a plot
    Args: 
        df = df w/ daily HI max for a given city
        year = year you want to plot against average
        start = start of plot in julian days (e.g 1 - 365/366)
        end = end of plot in julian days
    """

    # Deal with leap year
    if year % 4 !=0:
        df.drop(columns ='02.29', inplace = True)
    
    # Subset data
    start = start - 1 # zero indexing 
    subset = df.iloc[:,start:end]
    
    # HI Max for year
    hi_year = subset.loc[str(year)]
    
    # make 34-avg daily hi and std
    means = subset.mean(axis = 0)
    stds = subset.std(axis = 0)
    
    # make colums to date time
    cols = pd.to_datetime([str(year)+'.'+date for date in hi_year.index])
    
    return hi_year, means, cols, stds

In [50]:
# Find Heat Wave From All DATA
def select_city_year(df, city_id, year):
    "Quick search to find city and years within HI_STATS"
    df_out = df[(df['ID_HDC_G0'] == city_id) & (df['year'] == year)]
    
    return df_out

In [72]:
# open heat events data
events_fn = os.path.join('/home/CHIRTS/UEH-daily/stats/','himax406_STATS.json')
events = pd.read_json(events_fn, orient = 'split')

In [73]:
# [4417, 'Aleppo', 2010]  [9691, 'Kolkata', 1998] 
city = select_city_year(events, 4417, 2010)
city

,ID_HDC_G0,year,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax,UID
98,4417,2010,8,43.880742,3.280742,26.245933,"[2010.07.29, 2010.07.30, 2010.07.31, 2010.08.0...","[0.259825, 3.74098, 6.741, 4.10323, 3.29362, 2...","[40.859825, 44.34098, 47.341, 44.70323, 43.893...",UID-984386
99,4417,2010,7,43.248183,2.648183,18.537278,"[2010.08.10, 2010.08.11, 2010.08.12, 2010.08.1...","[0.5230600000000001, 2.408305, 2.615866, 4.168...","[41.12306, 43.008305, 43.215866, 44.768524, 45...",UID-984387
100,4417,2010,3,42.813595,2.213595,6.640785,"[2010.08.19, 2010.08.20, 2010.08.21]","[2.120367, 2.33311, 2.187308]","[42.720367, 42.93311, 42.787308]",UID-984388


In [96]:
# Select event and compare to long term means
# Kolkata 1998 UID-3010504, Aleppo 2010 UID-984387 and UID-984386
event_id = 'UID-984386'
event = city[city['UID'] == event_id]
tmax = list(event['tmax'])[0]
dates = list(event['event_dates'])[0]

In [97]:
# Look at event tmax 
max(tmax)

47.341

In [98]:
# Look at event dates 
len(dates)

8

In [99]:
#### Heat Index Data
data = 'himax'
data_label = '$WBGT_{max}$ '
DATA_IN = os.path.join('/home/CHIRTS/UEH-daily/','GHS-'+data+'/') # output from avg wbgt/hi max
FIG_OUT = '/home/cascade/projects/UrbanHeat/figures/'
DS = u"\N{DEGREE SIGN}"
t = 40.6 # wbgt (30) or hi (46.1) threshold

In [111]:
# Get long term means
# Args
#[4417, 'Aleppo'] 2010 [2046, 'Paris'] 2003 [9691, 'Kolkata'] 1998 ['6955, Dehli']

# Args
city_list = [4417, 'Aleppo']
year = 2010

# April 1 to Sep 30 (Use Julian Days), or 1 - 182 lagos
start = 91 
end = 273

# Make Data 
df = make_data(DATA_IN, 'ID_HDC_G0', city_list[0])
years, means, cols, stds = plot_data(df, year, start, end)

In [112]:
# what is the rank of HImax in the entire record?
sorted(df.to_numpy().flatten(), reverse = True)

[47.85866,
 47.341,
 47.222214,
 46.940273,
 46.791942999999996,
 46.568493,
 46.32212,
 46.1024,
 46.08296,
 45.971977,
 45.628246000000004,
 45.541016,
 45.514904,
 45.25997,
 45.241493,
 45.222027000000004,
 45.170605,
 45.103415999999996,
 44.85517,
 44.846848,
 44.83897,
 44.768524,
 44.70892,
 44.70323,
 44.667343,
 44.613037,
 44.59199,
 44.587105,
 44.505604,
 44.47729,
 44.474472,
 44.42072,
 44.368584000000006,
 44.34098,
 44.33639,
 44.325542,
 44.29978,
 44.281143,
 44.264297,
 44.217857,
 44.15063,
 44.143433,
 44.014526000000004,
 44.002525,
 43.979034000000006,
 43.92775,
 43.923206,
 43.89362,
 43.87218,
 43.851757,
 43.83255,
 43.79735,
 43.77262,
 43.769585,
 43.75018,
 43.747295,
 43.697174,
 43.694897,
 43.671425,
 43.66039,
 43.64654,
 43.63062,
 43.442122999999995,
 43.423527,
 43.42151,
 43.4075,
 43.402786,
 43.374565000000004,
 43.345303,
 43.33489,
 43.32069,
 43.320118,
 43.317935999999996,
 43.299335,
 43.293686,
 43.290577,
 43.262640000000005,
 43.215866,


In [113]:
cols

DatetimeIndex(['2010-04-01', '2010-04-02', '2010-04-03', '2010-04-04',
               '2010-04-05', '2010-04-06', '2010-04-07', '2010-04-08',
               '2010-04-09', '2010-04-10',
               ...
               '2010-09-21', '2010-09-22', '2010-09-23', '2010-09-24',
               '2010-09-25', '2010-09-26', '2010-09-27', '2010-09-28',
               '2010-09-29', '2010-09-30'],
              dtype='datetime64[ns]', length=183, freq=None)

In [115]:
# make a df
df = pd.DataFrame()
df['dates'] = cols
df['max']  = years.to_list()
df['avg'] = means.to_list()
df['dif'] = df['max'] - df['avg']

In [116]:
df

,dates,max,avg,dif
0,2010-04-01,20.215925,20.563679,-0.347754
1,2010-04-02,22.929924,19.757880,3.172044
2,2010-04-03,24.583519,20.418434,4.165085
3,2010-04-04,24.481691,20.370751,4.110940
4,2010-04-05,24.756367,21.742270,3.014097
...,...,...,...,...
178,2010-09-26,32.261818,31.668643,0.593175
179,2010-09-27,34.404118,31.765412,2.638706
180,2010-09-28,32.986008,31.068096,1.917912
181,2010-09-29,34.463043,30.896235,3.566808


In [117]:
type(df['dates'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [118]:
type(dates[0][0])

str

In [119]:
# start and end of heat wave
start = dates[0]
end = dates[-1]

In [120]:
out = df[(df['dates'] >= start) & (df['dates'] <= end)]

In [121]:
# peak heat wave
out.max()

dates    2010-08-05 00:00:00
max                   47.341
avg                  39.0942
dif                  9.05519
dtype: object

In [122]:
out[out['dif'] == out['dif'].max()]

,dates,max,avg,dif
121,2010-07-31,47.341,38.285811,9.055189


In [123]:
for dif in df['dif']:
    print(dif)

-0.34775358823529245
3.172043558823532
4.16508519117647
4.110940441176474
3.01409739705883
3.3695781176470554
-4.224510617647063
-5.494344808823531
-4.092693735294116
-2.8453546470588194
-4.516992970588234
-5.440255602941171
-3.036116117647058
-1.8543154411764675
1.8551093529411702
4.783531470588237
4.89916485294118
4.342953705882351
5.141150676470584
6.204444235294119
-0.8078437941176446
-1.9966462647058805
0.41618079411763986
2.1942171176470637
3.357165764705883
3.3922050882352934
1.2685657352941284
-6.6604488823529415
-3.689127411764705
-3.7089661764705824
-4.222438823529412
-2.069204882352942
0.5544949705882338
0.856665794117653
2.015017823529412
2.8824497647058784
3.594688411764711
2.9946388529411756
2.13490861764706
6.921765558823534
5.631298764705885
5.224262058823534
5.401429000000011
4.839913705882353
4.833134794117651
5.279025735294123
-0.43794702941175956
-1.9739935588235262
-4.875830000000004
-4.318485617647056
-2.7392007352941157
0.09009938235294257
-6.831568382352938
-4.9

In [124]:
df[df['dif'] > 8]

,dates,max,avg,dif
121,2010-07-31,47.341,38.285811,9.055189


In [125]:
len(df)

183

# Phoenix Caclulations
1990-6-26 Phoenix, AZ Weather History. https://www.wunderground.com/history/daily/KPHX/date/1990-6-26. Weather Underground. (June 10, 2021).<br>A. Minkler, At 118 degrees, Thursday heat in Phoenix breaks daily record set in 1934. The Arizona Republic (2020) (June 18, 2021).


In [ ]:
# Functions
def c_to_f(C):
    "Convert HI C to F"
    return 1.8 * C + 32 

def hi_to_wbgt(HI):
    """ Convert HI to WBGT using emprical relationship from Bernard and Iheanacho 2015
    WBGT [◦C] = −0.0034 HI2 + 0.96 HI−34; for HI [◦F]
    """
    
    WBGT = -0.0034*HI**2 + 0.96*HI - 34
    
    return WBGT

In [ ]:
# tmax was 122, rh was 11 using https://www.wpc.ncep.noaa.gov/html/heatindex.shtml
# then HI was 49 

# Check out cities

In [ ]:
def select_city_year(df, city_id, year):
    "Quick search to find city and years within HI_STATS"
    df_out = df[(df['ID_HDC_G0'] == city_id) & (df['year'] == year)]
    
    return df_out


In [ ]:
city = select_city_year(ALL_DATA, 4417, 2010)
city

# LA

In [ ]:
from glob import glob 
import os

In [ ]:
dir_list = '/home/cascade/projects/UrbanHeat/data/interim/ERA5_HI/'


In [ ]:
fn = 'GHS-ERA5-HI_2009.csv'
data = pd.read_csv(dir_list+fn)
city = data[data['ID_HDC_G0'] == 14]

In [ ]:
temps_list = []
dates_list = []
for fn in sorted(os.listdir(dir_list)):
    data = pd.read_csv(dir_list+fn)
    city = data[data['ID_HDC_G0'] == 14]
    dates = list(city.iloc[:,3:])
    temps = list(city.iloc[:,3:].values[0])
    dates_list.extend(dates)
    temps_list.extend(temps)

In [ ]:
len(dates_list)

# Africa

In [ ]:
#### Dependencies
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib.dates as mdates

In [ ]:
def make_data(dir_in, geog, location):
    """Function makes data to plot daily city-level HI Max and average
    Args:
        dir_in = directory to get data
        geog = column for geography, city-level = 'ID_HDC_G0'
        location = usually a city id
    """
    
    fn_list = sorted(glob.glob(dir_in+'*.csv')) # get data
    df_out = pd.DataFrame() # to write dataframe
    
     # get leap year cols from 2016
    hi16 = pd.read_csv(fn_list[33]) 
    cols = list(hi16.iloc[:,3:].columns)
    cols = [year[5:] for year in cols] # cols for data frame
    
    temp_list = [] # empty list for temps
    
    # loop through dir and get data
    for i, fn in enumerate(fn_list):
        df = pd.read_csv(fn) # open data frame
        year_label = [(df.columns[3]).split('.')[0]] # get year
        row = df[df[geog] == location]
        temp = row.iloc[:,3:] # get only temp columns
        
        # add in col for leap years
        if temp.shape[1] == 365:
            temp.insert(loc = 59, column = year_label[0]+'.02.29', value = np.nan, allow_duplicates=False)

        # Set Index & Columns
        temp.index = year_label
        temp.columns = cols # revalue to m.d
    
        # add to list
        temp_list.append(temp)
    
    df_out = pd.concat(temp_list) # make one big dataframe
    
    return df_out

In [ ]:
def plot_data(df, year, start, end):#, start, end):
    """ Make the data for a plot
    Args: 
        df = df w/ daily HI max for a given city
        year = year you want to plot against average
        start = start of plot in julian days (e.g 1 - 365/366)
        end = end of plot in julian days
    """

    # Deal with leap year
    if year % 4 !=0:
        df.drop(columns ='02.29', inplace = True)
    
    # Subset data
    start = start - 1 # zero indexing 
    subset = df.iloc[:,start:end]
    
    # HI Max for year
    hi_year = subset.loc[str(year)]
    
    # make 34-avg daily hi
    means = subset.mean(axis = 0)
    
    # make colums to date time
    cols = pd.to_datetime([str(year)+'.'+date for date in hi_year.index])
    
    return hi_year, means, cols

In [ ]:
# Args
#[3342, 'Cairo'] #[1910, 'Accra'] 3268, Cape Town
#[4417, 'Aleppo'] 2010 [2046, 'Paris'] 2003 [9691, 'Kolkata'] 1998 ['6955, Dehli']

# Args
city_list = [9691, 'Kolkata'] 
year = 1998
font_size = 10


# April 1 to Sep 30 (Use Julian Days), or 1 - 182 lagos
start = 91 
end = 273

# Labels
hi_label = str(year)+' '+data
labels =  ['avg. '+data, hi_label, str(t)+''+DS+'C']  # <<<<<------------ Be sure to update! 

# Make Data 
df = make_data(DATA_IN, 'ID_HDC_G0', city_list[0])
years, means, cols = plot_data(df, year, start, end)

# Trend First vs. Second Half of Data

In [ ]:
data.head()

In [ ]:
#### Share of exposure due to heat by 17 year split

## 1983 - 1999
data1 = HI_STATS[(HI_STATS['year'] >= 1983) & (HI_STATS['year'] < 2000)]
coef1pop , r21pop, p1pop  = lm_func(data1 , 'people_days_pop')
coef1heat , r21heat, p1heat = lm_func(data1 , 'people_days_heat')

years = list(np.unique(data1['year']))
plt.plot(years, data1.groupby('year')['people_days_heat'].sum())
sns.regplot(years, data1.groupby('year')['people_days_heat'].sum(), 
            color = 'blue', scatter = False, truncate = True)

## 2000 - 2016
data2 = HI_STATS[(HI_STATS['year'] >= 2000) & (HI_STATS['year'] <= 2016)]
coef2heat , r22heat, p2heat = lm_func(data2 , 'people_days_heat')
coef2pop , r22pop, p1pop  = lm_func(data2 , 'people_days_pop')

years = list(np.unique(data1['year']))
plt.plot(years, data2.groupby('year')['people_days_heat'].sum())
sns.regplot(years, data2.groupby('year')['people_days_heat'].sum(), 
            color = 'orange', scatter = False, truncate = True)

In [ ]:
## 2000 - 2016
data2pop = HI_STATS[(HI_STATS['year'] >= 1983) & (HI_STATS['year'] < 2000)]
coef2pop , r22pop, p1pop  = lm_func(data2pop , 'people_days_pop')

data2heat = HI_STATS[(HI_STATS['year'] >= 2000) & (HI_STATS['year'] <= 2016)]
coef2heat , r22heat, p2heat = lm_func(data2heat , 'people_days_heat')


In [ ]:
## Estimates
print('From 83 - 99, contribution from heat was', coef1heat/(coef1pop+coef1heat))
print('From 00 - 16, contribution from heat was', coef2heat/(coef2pop+coef2heat))
print('From 83 - 00, heat was', coef1heat/10**9, round(p1heat, 3))
print('From 00 - 16, heat was', coef2heat/10**9)
print('From 83 - 00, pop was', coef1pop/10**9)
print('From 00 - 16, pop was', coef2pop/10**9)